
# Game Generation

In this section, we describe how the initial conditions of the game are generated.


## Preliminaries

Let:

- $n$: number of agents.
- $I := \{1, \dots, n\}$
- $m$: number of goods.
- $G := \{1, \dots, m\}$
- $M$: money endowment.  
- $\mathbf{e} = \langle\mathbf{e}^1, \dots, \mathbf{e}^n\rangle$: list of good endowment vectors, one for each agent.
- $\mathbf{e}^i = \langle e^i_1, \dots, e^i_m\rangle$: list of good endowments for agent $i$.
- $\mathbf{s} = \langle \mathbf{s}^1, \dots, \mathbf{s}^n \rangle$: a list of preference score vectors, one for each agent.
- $\mathbf{s}^i = \langle s^i_1, \dots, s^i_m \rangle$: a list of preference scores, one for each good, for agent $i$.
- $\mathbf{x} = \langle \mathbf{x}^1, \dots, \mathbf{x}^n \rangle$: a list of good holding vectors, one for each agent.
- $\mathbf{x}^i = \langle x^i_1, \dots, x^i_m \rangle$: a list of good holdings, one for each good, for agent $i$.
- $\mathbf{p} = \langle p_1, \dots, p_m \rangle$: a list of prices, one for each good.
- $\mathbf{f} = \langle f^1, \dots, f^n\rangle$: list of money holdings, one for each agent.
- $f^i, x^i_j, s^i_j, e^i_j$ are non-negative reals.

In [1]:
import pprint
import random
from typing import List
import numpy as np
n = 3    # nb_agents
m = 5    # nb_goods
M = 20   # money
fractions = [1/i+2 for i in range(m)]  # type: List[int]
fractions = []

# type: List[List[int]]
S = [random.sample(utility_values, len(utility_values)) for _ in range(n)] 

print("Number of agents: ", n)
print("Number of goods: ", m)
print("Money endowment: ", M)
print("Utility values: ", pprint.pformat(utility_values))
print("Preference matrix ({} x {}): ".format(n, m), pprint.pformat(S))


ZeroDivisionError: division by zero


## Number of good instances

- $I_j$: number of instances of good $j$  
- $\mathbf{I} = \langle I_1, \dots, I_M\rangle$: vector of number of instances.   

In our setting, $I_j \sim \lfloor \mathcal{U}(a, b) \rceil$, where:

- $\Delta = n / g$  
- $a = n - \Delta$   
- $b = n + \Delta$  

The value $g$ is a parameter that can be tuned in order to get interesting configurations.   
$\Delta$ depends on the number of agents (the more agents we have, the wider the interval to sample from), 
but scaled by the parameter $g$. 

Code:


In [2]:

def sample_good_instance(n, g) -> int:
    """Sample the number of instances for a good.
    :param n: the number of agents
    :param g: the tuning parameter
    :return the number of instances I sampled.
    """
    delta = n/g
    a = n - delta
    b = n + delta
    # Return random integer in range [a, b]
    nb_instances = round(np.random.uniform(a, b))
    return nb_instances

g = 3  # the good tuning parameter
# type: List[int]
instances_per_good = [sample_good_instance(n, g) for _ in range(m)]

print("Number of agents: ", n)
print("Good tuning parameter g: ", g)
print("Minimum and Maximum: [{}, {}]".format(round(n - n/g), round(n + n/g)))
print("Instances per good: ", instances_per_good)


Number of agents:  3
Good tuning parameter g:  3
Minimum and Maximum: [2, 4]
Instances per good:  [3, 2, 2, 4, 4]



## Endowments

Let $E$ be a $(n \times m)$ matrix where 
each row $\mathbf{e}_i = \langle e_{i1}, \dots e_{im}\rangle$ tells how many
instances of good $1, \dots, m$ the agent $i$ holds. 

The problem of generating uniform endowments across agents for a given good can be seen as sampling from a 
multinomial distribution, with $I_j$ trials and $n$ possible outcomes.

In poor words:

- for every good $j = 1, \dots, m$:
    - for $I_j$ times:
        - select uniformly at random one of the $n$ agents and increase his endowment for good $j$. 

Code:


In [3]:

endowments = [[0] * m for _ in range(n)]  # type: List[List[int]]

for good_id in range(m):
    for _ in range(instances_per_good[good_id]):
        agent_id = random.randint(0, n - 1)
        endowments[agent_id][good_id] += 1

print("Endowments: ")
print(np.asarray(endowments))

Endowments: 
[[1 0 1 1 2]
 [0 1 1 2 1]
 [2 1 0 1 1]]



However, in order to have a maximum of `2` instances per good for every agent,
for a given good $j$, we do the following:

- Let $h = \lfloor I_j / 2 \rfloor$
    - if $n\mod 2 \neq 0$ use two values $h, h'$ such that $h' = h + 1$, otherwise $h = h'$  
- Let $\mathbf{a} \in \{0, 1\}^n$ be an allocation of good instances of $j$. Formally, $\mathbf{a}$ is a 
sample from a multivariate hypergeometric distribution.
    - In simpler terms, we choose an agent for $h$ times __without replacement__, and for every 
    chosen agent $i$ we have $a_i = 1$.  
    - in particular, it must be true that $\sum_i a_i = h$ and $\max_i a_i = 1$
- Let's compute two allocations $\mathbf{a}$ and $\mathbf{a}'$, associated with $h$ and $h'$ respectively.
- $\forall i$, we will have $e_{ij} = a_i + a_i'$ 

Repeat this procedure for every good $j = 1, \dots, m$.

The code: 


In [4]:

def compute_allocation(n: int, h: int) -> List[int]:
    """
    Compute an allocation (defined above).
    :param n: the number of agents.
    :param h: the number of instances to allocate.
    :return: the allocation (a vector of 0s and 1s
    """
    allocation = [0] * n
    for i in random.sample(range(n), h):
        allocation[i] = 1
    return allocation

def compute_endowment_of_good(n, nb_instances) -> List[int]:
    """
    Compute the allocation for all the agent of a single good.
    :param n: the number of agents.
    :param nb_instances: the number of instances of the good.
    :return: the endowment of good j for all the agents.
    """
    I_j = nb_instances
    h_1, h_2 = (I_j // 2, I_j // 2) if I_j % 2 == 0 else (I_j // 2, I_j // 2 + 1)
    a_1, a_2 = [compute_allocation(n, h_1), compute_allocation(n, h_2)]
    
    endowment = [a_1[idx] + a_2[idx] for idx in range(n)]
    
    return endowment

# matrix (m X n)
# type: List[List[int]]
endowments_by_good = [compute_endowment_of_good(n, I_j) for I_j in instances_per_good]

# matrix (n X m)
# type: List[List[int]]
endowments = np.asarray(endowments_by_good).T.tolist()

print("Endowments: ")
print(np.asarray(endowments))


Endowments: 
[[1 0 0 2 2]
 [2 1 1 0 0]
 [0 1 1 2 2]]
